# Prelude

In [ ]:
# print code version (hash of checked out version)
print("\nCurrent commit:")
!git log -1
print("\nChanges since last commit:")
!git status --short

# print date and time of script execution
import datetime
print("\nNotebook executed at at {} in following directory:".format(datetime.datetime.now()))
%cd ..

# Enable interactive plots (%matplotlib -l to list backends)
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D

from bgcellmodels.common import analysis, units, morphology, treeutils
import bgcellmodels.models.STN.GilliesWillshaw.gillies_pynn_model as gillies_model
import neuron; h = neuron.h
import bluepyopt.ephys as ephys

# Load custom synapse mechanisms
neuron.load_mechanisms('/home/luye/workspace/bgcellmodels/mechanisms/synapses')

# Create cell

In [ ]:
# ! EphysModelWrapper instantiates the cell in __init__()
cell = gillies_model.StnCellModel(
            calculate_lfp=True,
            lfp_sigma_extracellular=0.3,)
icell = cell.icell
nrnsim = cell.sim

In [ ]:
named_seclists =  {listname: list(getattr(icell, listname)) for listname in cell.seclist_names}
for k, v in named_seclists.items():
    if len(v)==0:
        named_seclists.pop(k) # don't include empty SectionLists
    else:
        print("{} : {} sections".format(k, len(v)))

somatic = named_seclists['somatic']
dendritic = named_seclists['basal']

soma = somatic[0]
dend = dendritic[0]

nseg = sum((sec.nseg for sec in icell.all))
print("Total number of compartments: {}".format(nseg))

# Give our cell 3D coordinates and shape info
h.define_shape(sec=soma)

In [ ]:
# test translation
def get_coordinate(i, sec):
    return [h.x3d(i, sec=sec), h.y3d(i, sec=sec), h.z3d(i, sec=sec)]

def translate_sec(sec, dvec):
    for i in range(int(h.n3d(sec=sec))):
        old_xyz = get_coordinate(i, sec)
        new_xyz = [sum(coord) for coord in zip(dvec, old_xyz)]
        h.pt3dchange(i, new_xyz[0], new_xyz[1], new_xyz[2], h.diam3d(i, sec=sec))

dend_start = get_coordinate(0, dend)
print(dend_start)

translate_sec(soma, [50.0, 50.0, 50.0])
h.define_shape(sec=soma)

dend_start = get_coordinate(0, dend)
print(dend_start)


## Synapse Locations

A suitable compartment can be selected either using `treeutils.subtree_topology` or by using 
NEURON GUI -> Tools -> Distributed Mechanisms > Viewers > Shape Name

## LFP Electrode

In [ ]:
import lfpsim

def get_center(sec):
    """
    Get coordinates of Section's center
    """
    n3d = int(h.n3d(sec=sec)) # number of 3D points associated with Section
    if n3d == 0:
        raise ValueError("Section has no 3D shape information associated. Call h.define_shape() first")
    i = 0
    loc_start = np.array([h.x3d(i, sec=sec), h.y3d(i, sec=sec), h.z3d(i, sec=sec)])
    i = n3d - 1
    loc_end = np.array([h.x3d(i, sec=sec), h.y3d(i, sec=sec), h.z3d(i, sec=sec)])
    center = (loc_start + loc_end) / 2
    return center

# Position the electrode 500 um above cell
sigma = extracellular_conductivity = 0.3

# coords = get_center(soma)
# coords[2] += 500.0 # 500 micron above center in z-direction
# coords = h.Vector(coords)
coords = h.Vector([50.0*100*0.5, 0.0, 500.0]) # same as in simulation

cvode = h.CVode()
cvode.use_fast_imem(True)

# METHOD A
# summator = h.insert_lfp_summator(icell.soma[0])
# h.add_lfp_sources(summator, True, "PSA", sigma, coords, icell.somatic, icell.basal)

# METHOD B
tracker = h.LfpTracker(icell.soma[0], True, "PSA", sigma, coords, icell.somatic, icell.basal)
summator = tracker.summator


# Excitatory + Inhibitory synapses

The Observations Database contains following observations that may help in configuring the distribution of synapses on the STN dendritic tree:

## Calibration Data

See notebook `calibrate_synapses.ipynb`

## Make Synapses

### Make GPe (GABA) Synapses

In [ ]:
from bgcellmodels.common.protocols import pick_random_segments

rng_seed = 8
highest_indices = {rng_seed: 0} # max high_index for each low_index (seed)
stim_data = {}
rec_secs = {}


# Filter to select proximal, larger-diam dendritic segments
num_gpe_syn = 8

# see Gillies cell diagram
# can_have_synapse = lambda seg: seg.diam > 1.0
h.distance(0, 0.5, sec=soma)
can_have_synapse = lambda seg: h.distance(1, seg.x, sec=seg.sec) < 120.0
target_segs = pick_random_segments(dendritic, num_gpe_syn, can_have_synapse)

# Make synapses
for i_seg, target_seg in enumerate(target_segs):
    
    tstart = 300
    stim_rate = 50.0 # discharge rate in between pauses
    stim_interval = stim_rate**-1*1e3
    pause_dur = 300.0 # duration of pauses
    discharge_dur = 3000.0 # duration of firing in between pauses
    dur_max_ms = 10000.0
    

    # RNG settings
    num_indep_repicks = dur_max_ms / stim_interval + 1000
    low_index = rng_seed
    highest_index = highest_indices.get(low_index, 0)
    high_index = int(highest_index + num_indep_repicks)
    highest_indices[low_index] = high_index # update highest index

    # MCellRan4: each stream should be statistically independent as long as 
    # the highindex values differ by more than the eventual length of the stream.
    # See http://www.neuron.yale.edu/neuron/static/py_doc/programming/math/random.html?highlight=MCellRan4
    stimrand = h.Random() # see CNS2014 Dura-Bernal example or EPFL cell synapses.hoc file
    stimrand.MCellRan4(high_index, low_index)
    stimrand.negexp(1) # if num arrivals is poisson distributed, ISIs are negexp-distributed

    # make bursting NetStim
    stimsource = h.BurstStim()
    stimsource.fast_invl = stim_interval
    stimsource.slow_invl = pause_dur
    stimsource.burst_len = discharge_dur*1e-3*stim_rate
    stimsource.start = tstart
    stimsource.noise = 1.0
    stimsource.noiseFromRandom(stimrand) # Set it to use this random number generator
    
    
    # GABA Synapse
    syn = h.GABAsyn(target_seg)
    nc = h.NetCon(stimsource, syn)
    nc.delay = 1.0
    nc.weight[0] = 1.0
    
    # Synapse parameters
    tau_rise = 2.0
    tau_decay = 6.0
    tau_rec = 17300.0
    tau_facil = 1.0
    P_release = 0.2
    hill_factor = 0.35
    
    peak_factor = 250 # 1 / P_release
    syn.gmax_GABAA = 7.0 * peak_factor * 1e-3 # conductance in [nS]
    syn.tau_r_GABAA = 2.0
    syn.tau_d_GABAA = 6.0
    
    peak_factor = 5 # 1 / hill_factor
    syn.gmax_GABAB = 7.0 * peak_factor * 1e-3
    syn.tau_r_GABAB = 5.0
    syn.tau_d_GABAB = 10.0
    
    syn.tau_rec = tau_rec
    syn.tau_facil = tau_facil
    syn.U1 = P_release
    
    # Save inputs
    stim_data.setdefault('NetStims', []).append(stimsource)
    stim_data.setdefault('RNGs', []).append(stimrand)
    stim_data.setdefault('GABA_NetCons', []).append(nc)
    stim_data.setdefault('GABA_synapses', []).append(syn)

# Print values - sanity check
last_syn = stim_data['GABA_synapses'][-1]
for attr in 'gmax_GABAA', 'gmax_GABAB':
    print("{} :\t\t{}".format(attr, getattr(last_syn, attr)))

### Make GLU (CTX) Synapses

In [ ]:
# Filter to select proximal, larger-diam dendritic segments
num_syn = 20
can_have_synapse = lambda seg: seg.diam <= 1.0 # see Gillies cell diagram
target_segs = pick_random_segments(dendritic, num_syn, can_have_synapse)

rec_secs = {}
stim_data['GLU_NetCons'] = []
stim_data['GLU_synapses'] = []

# Make synapses
for i_seg, target_seg in enumerate(target_segs):
    
    tstart = 300
    stim_rate = 50.0 # discharge rate in between pauses
    stim_interval = stim_rate**-1*1e3
    discharge_dur = 30.0 # duration of firing in between pauses
    pause_dur = 100.0 - discharge_dur # duration of pauses -> bursts @ ~ 10 Hz
    dur_max_ms = 10000.0
    

    # RNG settings
    num_indep_repicks = dur_max_ms / stim_interval + 1000
    low_index = rng_seed
    highest_index = highest_indices.get(low_index, 0)
    high_index = int(highest_index + num_indep_repicks)
    highest_indices[low_index] = high_index # update highest index

    # MCellRan4: each stream should be statistically independent as long as 
    # the highindex values differ by more than the eventual length of the stream.
    # See http://www.neuron.yale.edu/neuron/static/py_doc/programming/math/random.html?highlight=MCellRan4
    stimrand = h.Random() # see CNS2014 Dura-Bernal example or EPFL cell synapses.hoc file
    stimrand.MCellRan4(high_index, low_index)
    stimrand.negexp(1) # if num arrivals is poisson distributed, ISIs are negexp-distributed

    # make bursting NetStim
    stimsource = h.BurstStim()
    stimsource.fast_invl = stim_interval
    stimsource.slow_invl = pause_dur
    stimsource.burst_len = discharge_dur*1e-3*stim_rate # ((1.0/pause_rate_hz) - pause_dur) * stim_rate
    assert stimsource.burst_len > 0
    
    stimsource.start = tstart
    stimsource.noise = 1.0
    stimsource.noiseFromRandom(stimrand) # Set it to use this random number generator
    
    # Make GLU synapse
    syn = h.GLUsyn(target_seg)
    nc = h.NetCon(stimsource, syn)
    nc.delay = 1.0
    nc.weight[0] = 1.0
    
    # Common Synapse parameters
    syn.tau_rec = 200.0
    syn.tau_facil = 800.0
    syn.U1 = P_release = 0.1 # release probability
    
    # AMPA synapse parameters
    conductance_nS = 3.44
    peak_factor = 4.0
    syn.gmax_AMPA = conductance_nS * peak_factor * 1e-3 # gmax is in [uS]
    syn.tau_r_AMPA = 1.0
    syn.tau_d_AMPA = 4.0
    
    # NMDA synapse parameters
    conductance_nS = 7.0 # 7 nS
    peak_factor = 2.0
    syn.gmax_NMDA = conductance_nS * peak_factor * 1e-3
    syn.tau_r_NMDA = 3.7
    syn.tau_d_NMDA = 80
    
    # Save inputs
    stim_data.setdefault('NetStims', []).append(stimsource)
    stim_data.setdefault('RNGs', []).append(stimrand)
    stim_data.setdefault('GLU_NetCons', []).append(nc)
    stim_data.setdefault('GLU_synapses', []).append(syn)

# Print values - sanity check
last_syn = stim_data['GLU_synapses'][-1]
for attr in 'gmax_AMPA', 'gmax_NMDA':
    print("{} :\t\t{}".format(attr, getattr(last_syn, attr)))

### Plot Synapse Locations

In [ ]:
plt.figure()
ax = plt.subplot(111, projection='3d')
ax.view_init(90,90) # set azimut and elevation
# ax.set_zlim((-1,1)) # prevent rounding errors
# ax.set_axis_off() # hide x,y,z axes and grid

# Plot shape of our cell
h.define_shape(sec=soma) # for cells without 3D morphology
morphology.shapeplot(h, ax)

# Plot synapse locations
for i, syn in enumerate(stim_data['GLU_synapses']):
    seg = syn.get_segment()
    label = 'GLU' if i==0 else None
    morphology.mark_locations(h, seg.sec, seg.x, label=label, markspec='ob')

for i, syn in enumerate(stim_data['GABA_synapses']):
    seg = syn.get_segment()
    label = 'GABA' if i==0 else None
    morphology.mark_locations(h, seg.sec, seg.x, label=label, markspec='or')

plt.legend()
plt.show()

## Simulate & Analyze

### Record & Run

In [ ]:
# Define traces
rec_secs = {
    'soma': soma,
    'soma_spiker': h.NetCon(soma(0.5)._ref_v, None, -10.0, 0, 0),
    'lfp_sum': summator,
#     'prox': proximal_seg,
#     'mid': middle_seg,
#     'dist': distal_seg,
#     'synGABA': synapses[0],
}

trace_specs = {
    't_global': {'var': 't'},
    'V_soma': {'var':'v', 'sec':'soma', 'loc':0.5},
    'AP_soma': {'netcon': 'soma_spiker'},
    # LFP signal
    'LFP': {'pointp':'lfp_sum', 'var':'summed'},
    # Synapse variables
    'gAMPA': {'pointp':'synGLU0', 'var':'g_AMPA'},
    'gNMDA': {'pointp':'synGLU0', 'var':'g_NMDA'},
    'iAMPA': {'pointp':'synGLU0', 'var':'i_AMPA'},
    'iNMDA': {'pointp':'synGLU0', 'var':'i_NMDA'},
    'iGLU':  {'pointp':'synGLU0', 'var':'i'},
#     'Rrp': {'pointp':'synGLU', 'var':'R'},
#     'Use': {'pointp':'synGLU', 'var':'Use'},
    'gGABAA': {'pointp':'synGABA0', 'var':'g_GABAA'},
    'gGABAB': {'pointp':'synGABA0', 'var':'g_GABAB'},
    'iGABAA': {'pointp':'synGABA0', 'var':'i_GABAA'},
    'iGABAB': {'pointp':'synGABA0', 'var':'i_GABAB'},
    'iGABA':  {'pointp':'synGABA0', 'var':'i'},
}

# Record spikes at synapses
for i_syn, nc in enumerate(stim_data['GLU_NetCons']):
    con_label = "conGLU{}".format(i_syn)
    syn_label = "synGLU{}".format(i_syn)
    rec_secs[con_label] = nc
    rec_secs[syn_label] = nc.syn()
    trace_specs['AP_'+con_label] = {'netcon': con_label}
    trace_specs['i_'+syn_label] = {'pointp': syn_label, 'var':'i'}

for i_syn, nc in enumerate(stim_data['GABA_NetCons']):
    con_label = "conGABA{}".format(i_syn)
    syn_label = "synGABA{}".format(i_syn)
    rec_secs[con_label] = nc
    rec_secs[syn_label] = nc.syn()
    trace_specs['AP_'+con_label] = {'netcon': con_label}
    trace_specs['i_'+syn_label] = {'pointp': syn_label, 'var':'i'}

rec_dt = 0.05
vec_dict, markers = analysis.recordTraces(rec_secs, trace_specs, rec_dt)

In [ ]:
# Init and run simulation
h.dt = 0.025
h.celsius = 35.0
h.set_aCSF(4) # Hoc function defined in Gillies code
h.v_init = -68.0
h.tstop = 5000.0
h.init()

import time
tstart = time.time()
nrnsim.run(h.tstop, h.dt)
tstop = time.time()
cputime = tstop - tstart
num_segments = sum((sec.nseg for sec in h.allsec()))
print("Simulated {} segments for {} ms in {} s CPU time".format(
        num_segments, h.tstop, cputime))

### Plot Voltage Traces

In [ ]:
# Plot recorded traces
# t_global = vec_dict.pop('t_global').as_numpy() # inconsistent if dt != rec_dt 
interval = (2000, h.tstop)

# Plot somatic voltage traces
v_soma = vec_dict['V_soma'].as_numpy()
t_rec = np.arange(len(v_soma)) * rec_dt
# plt.figure()
# plt.plot(t_rec, v_soma)
# plt.grid(True)
spikes_soma = vec_dict['AP_soma'].as_numpy()
spike_range = (spikes_soma > interval[0]) & (spikes_soma <= interval[1])
soma_rate = len(spikes_soma[spike_range]) / float(interval[1]-interval[0]) * 1e3
analysis.plotTraces(vec_dict, rec_dt, interval, includeTraces=['V_soma'], title="Soma @ {} Hz".format(soma_rate))

### Plot Spike Trains

In [ ]:
# Plot spikes
spike_data = analysis.match_traces(vec_dict, lambda label: label.startswith('AP_conGLU'))
analysis.plotRaster(spike_data, interval, color='b', title='{} GLU spiketrains'.format(len(spike_data)))

spike_data = analysis.match_traces(vec_dict, lambda label: label.startswith('AP_conGABA'))
analysis.plotRaster(spike_data, interval, color='r', title='{} GABA spiketrains'.format(len(spike_data)))

### Plot Synaptic Currents

In [ ]:
# Plot synaptic variables
a = int(interval[0] // rec_dt)
b = int(interval[1] // rec_dt)
t_plot = t_rec[a:b]

# Plot GLU synapse variables
fig, axes = plt.subplots(2, 1, figsize=(10,6))

ax = axes[0]
ax.plot(t_plot, vec_dict['gAMPA'].as_numpy()[a:b], 'b', label='gAMPA')
ax.plot(t_plot, vec_dict['gNMDA'].as_numpy()[a:b], 'r', label='gNMDA')
ax.set_ylabel('conductance (uS)')
ax.legend()
ax.grid(True)
ax.set_title('Single GLU Synapse')

ax = axes[1]
ax.plot(t_plot, vec_dict['iAMPA'].as_numpy()[a:b], 'b', label='iAMPA')
ax.plot(t_plot, vec_dict['iNMDA'].as_numpy()[a:b], 'r', label='iNMDA')
ax.plot(t_plot, vec_dict['iGLU'].as_numpy()[a:b], 'g', label='i_tot')
ax.set_ylabel('current (nA)')
ax.legend()
ax.grid(True)

# Plot GABA synapse variables
fig, axes = plt.subplots(2, 1, figsize=(10,6))

ax = axes[0]
ax.plot(t_plot, vec_dict['gGABAA'].as_numpy()[a:b], 'b', label='gGABAA')
ax.plot(t_plot, vec_dict['gGABAB'].as_numpy()[a:b], 'r', label='gGABAB')
ax.set_ylabel('conductance (uS)')
ax.legend()
ax.grid(True)
ax.set_title('Single GABA Synapse')

ax = axes[1]
ax.plot(t_plot, vec_dict['iGABAA'].as_numpy()[a:b], 'b', label='iGABAA')
ax.plot(t_plot, vec_dict['iGABAB'].as_numpy()[a:b], 'r', label='iGABAB')
ax.plot(t_plot, vec_dict['iGABA'].as_numpy()[a:b], 'g', label='i_tot')
ax.set_ylabel('current (nA)')
ax.grid(True)
ax.legend()

# Plot total GABA synapse current
itot_GABA = vec_dict['i_synGABA0'].as_numpy()
for i in range(1, len(stim_data['GABA_synapses'])):
    itot_GABA += vec_dict['i_synGABA{}'.format(i)].as_numpy()

itot_GLU = vec_dict['i_synGLU0'].as_numpy() * -1
for i in range(1, len(stim_data['GLU_synapses'])):
    itot_GLU -= vec_dict['i_synGLU{}'.format(i)].as_numpy()

plt.figure(figsize=(10,4))
plt.plot(t_rec, itot_GABA, 'r', label='i_GABA')
plt.plot(t_rec, itot_GLU, 'b', label='i_GLU')
plt.ylabel('current (nA)')
plt.xlabel('time (ms)')
plt.legend()
plt.grid()
plt.show()
plt.suptitle('Total GABA and GLU current')
# Plot remaining traces
# figs_vm = analysis.plotTraces(vec_dict, rec_dt, traceSharex=True) # yRange=(-80,40),

### Plot LFP Signal

In [ ]:
# Plot GABA synapse variables
fig, ax = plt.subplots(figsize=(10,6))

ax.plot(t_plot, vec_dict['LFP'].as_numpy()[a:b], 'b', label='LFP')
ax.set_ylabel('LFP (nV)')
ax.legend()
ax.grid(True)
ax.set_title('LFP Signal')